In [71]:
import pandas as pd
import numpy as np
import math 

Read the book sent over by Westley and Stephanie

In [72]:
dat_orig = pd.read_excel("20190624_sales_fcst.xlsx", header = 2)
dat_orig.fillna(method='ffill', inplace=True)
dat_orig = dat_orig.filter(regex=r'(Wk.|Unnamed: 4|Report in thousands|Unnamed: 0)')
dat_orig.head(50)

,Unnamed: 0,Report in thousands,Report in thousands.1,Unnamed: 4,Feb Wk. 1 2019,Feb Wk. 2 2019,Feb Wk. 3 2019,Feb Wk. 4 2019,Mar Wk. 5 2019,Mar Wk. 6 2019,...,Nov Wk. 43 2019,Dec Wk. 44 2019,Dec Wk. 45 2019,Dec Wk. 46 2019,Dec Wk. 47 2019,Dec Wk. 48 2019,Jan Wk. 49 2019,Jan Wk. 50 2019,Jan Wk. 51 2019,Jan Wk. 52 2019
0,TOTAL,Sls Total Rtl,Total Company Operating Sales,Actual (TY) / Forecast,9.566299e+06,8.410535e+06,8.313224e+06,1.369493e+07,1.100258e+07,1.383237e+07,...,1.854832e+07,2.145241e+07,1.504820e+07,1.283804e+07,9.994588e+06,9.023529e+06,1.136200e+07,7.462384e+06,8.216790e+06,8.711446e+06
1,TOTAL,Sls Total Rtl,Total Company Operating Sales,Forecast,1.042099e+07,8.951728e+06,8.743079e+06,1.439291e+07,1.362358e+07,1.312598e+07,...,1.854832e+07,2.145241e+07,1.504820e+07,1.283804e+07,9.994588e+06,9.023529e+06,1.136200e+07,7.462384e+06,8.216790e+06,8.711446e+06
2,TOTAL,Sls Total Rtl,Total Company Operating Sales,Plan,1.095530e+07,9.652871e+06,9.992850e+06,1.377227e+07,1.437545e+07,1.487448e+07,...,1.897964e+07,2.170684e+07,1.541663e+07,1.347188e+07,1.012432e+07,9.379631e+06,7.845365e+06,1.163610e+07,9.331028e+06,7.787172e+06
3,TOTAL,Sls Total Rtl,Total Company Operating Sales,Var to Plan,-1.389004e+06,-1.242336e+06,-1.679626e+06,-7.734574e+04,-3.372870e+06,-1.042113e+06,...,-4.313250e+05,-2.544350e+05,-3.684284e+05,-6.338377e+05,-1.297361e+05,-3.561021e+05,3.516633e+06,-4.173716e+06,-1.114238e+06,9.242739e+05
4,TOTAL,Sls Total Rtl,Total Company Operating Sales,Target,9.566299e+06,8.410535e+06,8.313224e+06,1.369493e+07,1.100258e+07,1.383237e+07,...,1.866174e+07,2.155443e+07,1.513883e+07,1.292172e+07,1.005694e+07,9.081421e+06,1.143024e+07,7.496867e+06,8.206244e+06,9.148089e+06
5,TOTAL,Sls Total Rtl,Total Company Operating Sales,LY,8.206939e+06,9.840579e+06,1.014325e+07,1.229016e+07,1.518833e+07,1.338827e+07,...,2.056473e+07,1.440198e+07,1.315610e+07,1.254588e+07,1.037772e+07,8.978334e+06,1.102267e+07,7.478557e+06,7.597383e+06,7.402133e+06
6,TOTAL,Sls Total Rtl,Total Company Operating Sales,LLY,9.110087e+06,9.401258e+06,9.277343e+06,1.279341e+07,1.451657e+07,1.347729e+07,...,1.985019e+07,1.410739e+07,1.322949e+07,1.229943e+07,1.043165e+07,1.115279e+07,1.049968e+07,1.048615e+07,6.264101e+06,7.491346e+06
7,TOTAL,Sls Total Rtl,Total Company Operating Sales,Actual to Plan %,-1.267882e-01,-1.287012e-01,-1.680828e-01,-5.616047e-03,-2.346271e-01,-7.006046e-02,...,-2.272566e-02,-1.172142e-02,-2.389812e-02,-4.704895e-02,-1.281430e-02,-3.796547e-02,4.482435e-01,-3.586868e-01,-1.194122e-01,1.186919e-01
8,TOTAL,Sls Total Rtl,Total Company Operating Sales,Actual to LY %,1.656354e-01,-1.453211e-01,-1.804184e-01,1.143004e-01,-2.755903e-01,3.317060e-02,...,-9.805173e-02,4.895459e-01,1.438195e-01,2.328765e-02,-3.691868e-02,5.033750e-03,3.078496e-02,-2.162558e-03,8.152903e-02,1.768833e-01
9,TOTAL,Sls Total FP Rtl,Company Full Price Operating Sales,Actual (TY) / Forecast,8.210753e+06,6.626589e+06,5.647718e+06,1.101918e+07,9.178044e+06,1.195891e+07,...,1.630414e+07,1.883665e+07,1.282273e+07,8.734037e+06,5.064751e+06,4.343581e+06,7.465592e+06,4.397840e+06,4.006535e+06,6.753756e+06


In [73]:
h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
h2 = [x+1 for x in h1]

months = h1 + h2
weeks = list(range(1, 53))

month_map = pd.DataFrame()
month_map['FISCAL_MO'] = months
month_map['FISCAL_WK'] = weeks



In [80]:
def format_sales(df):
    
    ids = list(df.iloc[:, np.r_[0:1, 2:4]].columns)
    vals = list(df.iloc[:,4:].columns)
    
    melt = pd.melt(df, id_vars = ids, value_vars = vals, var_name = 'FISCAL_WK', value_name = 'SALES')
    
    melt.rename(columns={'Unnamed: 0':'Channel',
                      'Report in thousands.1': 'sales_segment',
                          'Unnamed: 4':'summation_type'
                          }, 
                 inplace=True)
    
    sub = melt[melt['summation_type'] == "Actual (TY) / Forecast"]
    sub = sub[sub['sales_segment'].isin(['Direct Net Sales', 'Retail Sales (Stores & Concierge)'])]
    sub[['Channel', 'SALES']].groupby('Channel').agg('sum')
    
    sub.FISCAL_WK = sub.FISCAL_WK.str[8:]
    sub.FISCAL_WK = sub.FISCAL_WK.str[:2]
    sub.FISCAL_WK = sub.FISCAL_WK.str.strip()
    
    h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
    h2 = [x+6 for x in h1]

    months = h1 + h2
    weeks = list(range(1, 53))
    qtrs = [math.ceil(x / 3) for x in months]

    month_map = pd.DataFrame()
    month_map['FISCAL_MO'] = months
    month_map['FISCAL_WK'] = weeks
    month_map['FISCAL_QTR'] = qtrs
    
    sub.FISCAL_WK = sub.FISCAL_WK.astype(str).astype(int)
    sales_fcst = pd.merge(sub, month_map, on='FISCAL_WK', how='inner')
    fcst = sales_fcst.query('summation_type == "Actual (TY) / Forecast"')
    
    fcst.Channel = np.where(fcst['Channel']=='Direct', 'D', 'R')
    
    return fcst

In [81]:
fcst = format_sales(dat_orig)
fcst.head()

,Channel,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO,FISCAL_QTR
0,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1,1
1,D,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1,1
2,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1,1
3,D,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1,1
4,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1,1


In [82]:
fcst.SALES.sum()

680155158.749575

In [83]:
qtr_data = pd.read_csv('quarterly_sales_to_custs.csv').sort_values('Channel')
qtr_data.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405
39,2018,D,5.933927e+07,447606,4,6.467598e+06,67558,2,0.108994,0.150932,1.077873e+06,5712,React__Omni,_Omni,6.856067e+06,63402,0.157214,0.090092,188.703201
38,2018,D,5.933927e+07,447606,4,6.467598e+06,67558,2,0.108994,0.150932,5.778195e+06,57690,React_1,1,6.856067e+06,63402,0.842786,0.909908,100.159382
37,2018,D,5.602950e+07,377032,3,6.236244e+06,55832,2,0.111303,0.148083,1.010387e+06,4236,React__Omni,_Omni,6.553415e+06,52434,0.154177,0.080787,238.523885
36,2018,D,5.602950e+07,377032,3,6.236244e+06,55832,2,0.111303,0.148083,5.543028e+06,48198,React_1,1,6.553415e+06,52434,0.845823,0.919213,115.005353


In [84]:
fcst.head()

,Channel,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO,FISCAL_QTR
0,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1,1
1,D,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1,1
2,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1,1
3,D,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1,1
4,R,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1,1


In [87]:
direct_total = fcst[['Channel', 'SALES', 'FISCAL_QTR']].query('Channel == "D"').groupby(['FISCAL_QTR', 'Channel']).agg('sum').reset_index()
retail_total = fcst[['Channel', 'SALES', 'FISCAL_QTR']].query('Channel == "R"').groupby(['FISCAL_QTR', 'Channel']).agg('sum').reset_index()
qtr_sales = pd.concat([direct_total, retail_total])

In [88]:
qtr_sales.SALES.sum()

680155158.7495749

In [95]:
qtr_sales = qtr_sales.rename(index=str, columns={"SALES": "channel_qtr_sales"})

In [96]:
qtr_dat = pd.merge(qtr_data, qtr_sales, on = ['FISCAL_QTR', 'Channel'], how = 'inner')
qtr_dat.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,chan_rec_seg_sales,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,3.297407e+06,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405,70927548.79
1,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,1.074583e+06,4675,React__Omni,_Omni,6.781020e+06,56169,0.158469,0.083231,229.857276,70927548.79
2,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,5.706438e+06,51494,React_1,1,6.781020e+06,56169,0.841531,0.916769,110.817524,70927548.79
3,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,1.807705e+07,52128,Existing__Omni,_Omni,5.305491e+07,262268,0.340723,0.198759,346.781892,70927548.79
4,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,3.497786e+07,210140,Existing_1,1,5.305491e+07,262268,0.659277,0.801242,166.450267,70927548.79


In [97]:
qtr_dat['rec_seg_exp_sales'] = qtr_dat.sales_pct * qtr_dat.channel_qtr_sales * qtr_dat.chan_rec_sales_pct
qtr_dat.head()

,FISCAL_YR,Channel,total_net_sales,total_custs,FISCAL_QTR,recency_net_sales_total,recency_cust_total,Recency,chan_rec_sales_pct,chan_rec_cust_pct,...,chan_rec__seg_custs,Customer_Type,Segment,chan_rec_sales,chan_rec__custs,sales_pct,cust_pct,d_p_cust,channel_qtr_sales,rec_seg_exp_sales
0,2018,D,5.721810e+07,401995,1,3.675641e+06,36585,0,0.064239,0.091009,...,31814,NTB_1,1,3.877800e+06,34250,0.850329,0.928876,103.646405,70927548.79,3.874375e+06
1,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,...,4675,React__Omni,_Omni,6.781020e+06,56169,0.158469,0.083231,229.857276,70927548.79,1.259988e+06
2,2018,D,5.721810e+07,401995,1,6.414162e+06,60238,2,0.112100,0.149848,...,51494,React_1,1,6.781020e+06,56169,0.841531,0.916769,110.817524,70927548.79,6.691007e+06
3,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,...,52128,Existing__Omni,_Omni,5.305491e+07,262268,0.340723,0.198759,346.781892,70927548.79,1.990510e+07
4,2018,D,5.721810e+07,401995,1,4.712819e+07,305170,1,0.823659,0.759139,...,210140,Existing_1,1,5.305491e+07,262268,0.659277,0.801242,166.450267,70927548.79,3.851501e+07


In [98]:
qtr_dat['implied_custs'] = qtr_dat.rec_seg_exp_sales / qtr_dat.d_p_cust
qtr_dat.implied_custs.sum()

3494822.550392436

In [100]:
qtr_dat[['FISCAL_QTR', 'implied_custs']].groupby('FISCAL_QTR').agg('sum')

,implied_custs
FISCAL_QTR,
1,844409.801298
2,875244.186943
3,834323.721593
4,940844.840559


In [101]:
qtr_dat.to_pickle('quarter_2019')

In [15]:
omni = year_end_fcst.query('Segment == "_Omni"')
omni

,Channel,Recency,Customer_Type,Segment,implied_custs
1,D,0,NTB__Omni,_Omni,20520.120222
5,D,1,Existing__Omni,_Omni,161649.139806
9,D,2,React__Omni,_Omni,40838.961381
3,R,0,NTB__Omni,_Omni,18863.811405
7,R,1,Existing__Omni,_Omni,160016.121752
11,R,2,React__Omni,_Omni,36733.887016


In [16]:
noms = pd.merge(omni, omni, on = 'Recency', how='inner')
noms = noms

In [19]:
noms

,Channel_x,Recency,Customer_Type_x,Segment_x,implied_custs_x,Channel_y,Customer_Type_y,Segment_y,implied_custs_y
0,D,0,NTB__Omni,_Omni,20520.120222,D,NTB__Omni,_Omni,20520.120222
1,D,0,NTB__Omni,_Omni,20520.120222,R,NTB__Omni,_Omni,18863.811405
2,R,0,NTB__Omni,_Omni,18863.811405,D,NTB__Omni,_Omni,20520.120222
3,R,0,NTB__Omni,_Omni,18863.811405,R,NTB__Omni,_Omni,18863.811405
4,D,1,Existing__Omni,_Omni,161649.139806,D,Existing__Omni,_Omni,161649.139806
5,D,1,Existing__Omni,_Omni,161649.139806,R,Existing__Omni,_Omni,160016.121752
6,R,1,Existing__Omni,_Omni,160016.121752,D,Existing__Omni,_Omni,161649.139806
7,R,1,Existing__Omni,_Omni,160016.121752,R,Existing__Omni,_Omni,160016.121752
8,D,2,React__Omni,_Omni,40838.961381,D,React__Omni,_Omni,40838.961381
9,D,2,React__Omni,_Omni,40838.961381,R,React__Omni,_Omni,36733.887016


In [37]:
oms = noms.query('Channel_x != Channel_y').copy()
oms['omni'] = oms.implied_custs_x + oms.implied_custs_y
oms = oms[['Customer_Type_x', 'omni']].copy()
oms = oms.drop_duplicates()

In [38]:
oms.omni.sum()

438622.0415827384

In [40]:
oms.head()

,Customer_Type_x,omni
1,NTB__Omni,39383.931627
5,Existing__Omni,321665.261558
9,React__Omni,77572.848397


In [47]:
melt.rename(columns={'Unnamed: 0':'MASTER_CHANNEL',
                      'Report in thousands.1': 'sales_segment',
                          'Unnamed: 4':'summation_type'
                          }, 
                 inplace=True)

We will do a quick aggregation at this point to confirm that the summations we've applied are working properly. We want to verify that these totals line up with the totals taken from Wes's workbook. The grand total should match the current sales forecast.

In [48]:
sub = melt[melt['summation_type'] == "Actual (TY) / Forecast"]
sub = sub[sub['sales_segment'].isin(['Direct Net Sales', 'Retail Sales (Stores & Concierge)'])]
sub.head()
sub[['MASTER_CHANNEL', 'SALES']].groupby('MASTER_CHANNEL').agg('sum')

,SALES
MASTER_CHANNEL,
Direct,2.782062e+08
RETAIL,4.019490e+08


In [35]:
sub.SALES.sum()

680155158.749575

In [36]:
sub.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES
54,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 1 2019,5476461.31
135,Direct,Direct Net Sales,Actual (TY) / Forecast,Feb Wk. 1 2019,4089837.30
243,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 2 2019,4662842.01
324,Direct,Direct Net Sales,Actual (TY) / Forecast,Feb Wk. 2 2019,3747693.15
432,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,Feb Wk. 3 2019,5506713.02


Now the data is formatted appropriately so we are ready to move with the next step. Ultimately we need our forecasts at the monthly level, so our next task is to clean up the `FISCAL_WK` column so that we can use to to create a `FISCAL_MO` column.

In [49]:
sub.FISCAL_WK = sub.FISCAL_WK.str[8:]
sub.FISCAL_WK = sub.FISCAL_WK.str[:2]
sub.FISCAL_WK = sub.FISCAL_WK.str.strip()
sub.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES
54,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31
135,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30
243,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01
324,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15
432,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02


In [50]:
sub.FISCAL_WK.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52'], dtype=object)

In [51]:
h1 = [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6]
h2 = [x+1 for x in h1]

months = h1 + h2
weeks = list(range(1, 53))

month_map = pd.DataFrame()
month_map['FISCAL_MO'] = months
month_map['FISCAL_WK'] = weeks

month_map.head()

,FISCAL_MO,FISCAL_WK
0,1,1
1,1,2
2,1,3
3,1,4
4,2,5


In [52]:
month_map.shape

(52, 2)

In [53]:
sub.dtypes

MASTER_CHANNEL     object
sales_segment      object
summation_type     object
FISCAL_WK          object
SALES             float64
dtype: object

In [54]:
sub.FISCAL_WK = sub.FISCAL_WK.astype(str).astype(int)
sales_fcst = pd.merge(sub, month_map, on='FISCAL_WK', how='inner')
sales_fcst.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1


In [55]:
sales_fcst.head(26)

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1
5,Direct,Direct Net Sales,Actual (TY) / Forecast,3,2806510.55,1
6,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,4,7207303.66,1
7,Direct,Direct Net Sales,Actual (TY) / Forecast,4,6487624.60,1
8,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,5,6706428.53,2
9,Direct,Direct Net Sales,Actual (TY) / Forecast,5,4296148.16,2


We might want to work with the LY , plan, or other summation types, but for this notebook we are only interested in the `Actual (TY) / Forecast` sales totals. So now we are ready to subset this data to just forecasted sales so we can get to work on building together a customer forecast.

In [58]:
fcst = sales_fcst.query('summation_type == "Actual (TY) / Forecast"')
fcst.head()

,MASTER_CHANNEL,sales_segment,summation_type,FISCAL_WK,SALES,FISCAL_MO
0,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,1,5476461.31,1
1,Direct,Direct Net Sales,Actual (TY) / Forecast,1,4089837.30,1
2,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,2,4662842.01,1
3,Direct,Direct Net Sales,Actual (TY) / Forecast,2,3747693.15,1
4,RETAIL,Retail Sales (Stores & Concierge),Actual (TY) / Forecast,3,5506713.02,1


In [57]:
fcst.shape

(104, 3)